# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [2]:
import napari
from macrohet import dataio, tile, visualise
import os, glob

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [79]:
%%time
expt_ID = 'ND0002'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
base_dir = f'/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/ND0002__2023-11-30T17_19_20-Measurement 2/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
# temporary hack to fix URL from incorrectly exported metadata
metadata['URL'] = metadata.apply(dataio.generate_url, axis=1)
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/388620 [00:00<?, ?it/s]

Extracting metadata complete!
CPU times: user 1min 21s, sys: 4.37 s, total: 1min 26s
Wall time: 3min 39s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0103K1F1P1R1,Ok,r01c03f01p01-ch1sk1fk1fl1.tiff,1,3,1,1,0,1,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.49+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
1,0103K1F1P1R2,Ok,r01c03f01p01-ch2sk1fk1fl1.tiff,1,3,1,1,0,2,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.723+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
2,0103K1F1P2R1,Ok,r01c03f01p02-ch1sk1fk1fl1.tiff,1,3,1,2,0,1,1,...,0,0.135468394,0,2023-11-30T17:22:10.067+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
3,0103K1F1P2R2,Ok,r01c03f01p02-ch2sk1fk1fl1.tiff,1,3,1,2,0,2,1,...,0,0.135468394,0,2023-11-30T17:22:10.287+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
4,0103K1F1P3R1,Ok,r01c03f01p03-ch1sk1fk1fl1.tiff,1,3,1,3,0,1,1,...,2E-06,0.135470405,0,2023-11-30T17:22:10.627+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388615,0612K150F9P1R2,Ok,r06c12f09p01-ch2sk150fk1fl1.tiff,6,12,9,1,149,2,1,...,-2E-06,0.1351538,268191.66,2023-12-03T20:06:16.08+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388616,0612K150F9P2R1,Ok,r06c12f09p02-ch1sk150fk1fl1.tiff,6,12,9,2,149,1,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.423+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388617,0612K150F9P2R2,Ok,r06c12f09p02-ch2sk150fk1fl1.tiff,6,12,9,2,149,2,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.657+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388618,0612K150F9P3R1,Ok,r06c12f09p03-ch1sk150fk1fl1.tiff,6,12,9,3,149,1,1,...,2E-06,0.135157794,268191.66,2023-12-03T20:06:17+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."


### checking if any files are missing in image directory

In [85]:
from tqdm.auto import tqdm

In [86]:
missing_files = []
for fn in tqdm(metadata['URL'], total = len(metadata['URL'])):
    path = os.path.join(base_dir, f'acquisition/ND0002__2023-11-30T17_19_20-Measurement 2/Images/{fn}')
    if not os.path.exists(path):
        missing_files.append(fn)

  0%|          | 0/388620 [00:00<?, ?it/s]

In [88]:
%%time
expt_ID = 'ND0002'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'
base_dir = f'/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
# temporary hack to fix URL from incorrectly exported metadata
# metadata['URL'] = metadata.apply(dataio.generate_url, axis=1)
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/388620 [00:00<?, ?it/s]

Extracting metadata complete!
CPU times: user 1min 37s, sys: 4.7 s, total: 1min 42s
Wall time: 10min 21s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0103K1F1P1R1,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,1,3,1,1,0,1,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.49+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
1,0103K1F1P1R2,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,1,3,1,1,0,2,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.723+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
2,0103K1F1P2R1,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,1,3,1,2,0,1,1,...,0,0.135468394,0,2023-11-30T17:22:10.067+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
3,0103K1F1P2R2,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,1,3,1,2,0,2,1,...,0,0.135468394,0,2023-11-30T17:22:10.287+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
4,0103K1F1P3R1,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,1,3,1,3,0,1,1,...,2E-06,0.135470405,0,2023-11-30T17:22:10.627+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388615,0612K150F9P1R2,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,6,12,9,1,149,2,1,...,-2E-06,0.1351538,268191.66,2023-12-03T20:06:16.08+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388616,0612K150F9P2R1,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,6,12,9,2,149,1,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.423+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388617,0612K150F9P2R2,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,6,12,9,2,149,2,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.657+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388618,0612K150F9P3R1,Ok,http://harmony-pc/ODA/Images/C/2666d92c-c046-4...,6,12,9,3,149,1,1,...,2E-06,0.135157794,268191.66,2023-12-03T20:06:17+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [ ]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

### Define row and column of choice

In [80]:
acq_ID = row, column = (3, 4)

### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [5]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [7]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
images

CPU times: user 1.13 s, sys: 7.42 ms, total: 1.13 s
Wall time: 1.32 s


dask.array<max-aggregate, shape=(150, 2, 6048, 6048), dtype=uint16, chunksize=(1, 2, 2160, 2160), chunktype=numpy.ndarray>

In [10]:
%%time
images = images.compute().compute()

FileNotFoundError: The file '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/ND0002/acquisition/Images/r03c04f01p01-ch1sk130fk1fl1.tiff' does not exist.

In [24]:
os.path.exists('/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/ND0002/acquisition/Images/r03c04f01p01-ch1sk1fk1fl1.tiff')

True

In [25]:
len(glob.glob(image_dir+'/*.tiff'))

307722

In [ ]:
388620

In [27]:
307722/388620

0.791832638567238

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/napari/_qt/widgets/qt_dims_slider.py", line 550, in mouseReleaseEvent
    self._on_click()
  File "/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/napari/_qt/widgets/qt_dims_slider.py", line 558, in _on_click
    return qt_dims.stop()
  File "/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/napari/_qt/widgets/qt_dims.py", line 321, in stop
    self._animation_worker._stop()
  File "/home/dayn/miniconda3/envs/brassica/lib/python3.9/site-packages/superqt/utils/_ensure_thread.py", line 156, in _func
    func_, self.thread(), await_return, timeout, self, *args, **kwargs
RuntimeError: wrapped C/C++ object of type AnimationWorker has been deleted


In [26]:
len(metadata)

388620

In [11]:
images.shape

(150, 2, 6048, 6048)

In [12]:
viewer = napari.Viewer()

viewer.add_image(images, channel_axis = 1)

[<Image layer 'Image' at 0x7f0170c426a0>,
 <Image layer 'Image [1]' at 0x7f0148352a00>]

In [10]:
print()

## Currently missing some images

due to failed export (only 80pc)

In [31]:
%%time
cut_images = images[0:119].compute().compute()

CPU times: user 4h 56min 19s, sys: 56min, total: 5h 52min 20s
Wall time: 1h 12min 18s


In [33]:
viewer = napari.Viewer(title = 'cropped')

viewer.add_image(cut_images, channel_axis = 1)

[<Image layer 'Image' at 0x7efdc1ed1370>,
 <Image layer 'Image [1]' at 0x7efdc02f9d30>]

# Trying two channel segmentation

In [43]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

# model = models.Cellpose(gpu=True, model_type='cyto')
model_path = '/mnt/DATA/macrohet/upstream_development/segmentation/cellpose_training/models/models/macrohet_seg'
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)
def segment(img, diameter = 30, channels = [0,0]):
    masks, flows, styles, diams = model.eval(img, diameter=diameter, channels=channels,
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Thu Jan  4 15:27:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:65:00.0  On |                  Off |
| 30%   47C    P8              35W / 300W |   3352MiB / 49140MiB |      0%      D

INFO:cellpose.core:** TORCH CUDA version installed and working. **


>>> GPU activated? YES


INFO:cellpose.models:>> cyto << model set to be used
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [36]:
img = images[0]

In [37]:
img.shape

(2, 6048, 6048)

In [44]:
masks = segment(img, diameter = None)

ValueError: not enough values to unpack (expected 4, got 3)

In [57]:
model = models.Cellpose(gpu=True, model_type='cyto')
output = model.eval(img, diameter = 300)#, diameter=diameter, channels=channels,
                                             # flow_threshold=None, cellprob_threshold=0)

INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>> cyto << model set to be used
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 112.45 sec


In [58]:
output[0].shape
masks = output[0]

In [59]:
viewer.add_labels(masks)

<Labels layer 'masks [3]' at 0x7effa00f3d90>

In [48]:
print()

In [72]:
img

dask.array<getitem, shape=(2, 6048, 6048), dtype=uint16, chunksize=(2, 2160, 2160), chunktype=numpy.ndarray>

# Trying a new segmentation approach

In [69]:
import torch
print(torch.__version__)

2.1.1+cu121


In [71]:
from segment_anything import SamPredictor, sam_model_registry


ModuleNotFoundError: No module named 'torchvision.ops'

In [77]:
import numpy as np

In [78]:
np.save('t0_ND2.npy', cut_images[0])